In [1]:
import oqs

from Crypto.PublicKey import ECC, RSA
from Crypto.Signature import eddsa, pkcs1_15, pss
from Crypto.Hash import SHA256
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto import Random
from Crypto.Random import get_random_bytes

from numpy import mean

from tabulate import tabulate

import time

In [2]:
kems = oqs.get_enabled_KEM_mechanisms()
print(kems)

['BIKE-L1', 'BIKE-L3', 'Classic-McEliece-348864', 'Classic-McEliece-348864f', 'Classic-McEliece-460896', 'Classic-McEliece-460896f', 'Classic-McEliece-6688128', 'Classic-McEliece-6688128f', 'Classic-McEliece-6960119', 'Classic-McEliece-6960119f', 'Classic-McEliece-8192128', 'Classic-McEliece-8192128f', 'HQC-128', 'HQC-192', 'HQC-256', 'Kyber512', 'Kyber768', 'Kyber1024', 'Kyber512-90s', 'Kyber768-90s', 'Kyber1024-90s', 'ntrulpr653', 'ntrulpr761', 'ntrulpr857', 'ntrulpr1277', 'sntrup653', 'sntrup761', 'sntrup857', 'sntrup1277', 'FrodoKEM-640-AES', 'FrodoKEM-640-SHAKE', 'FrodoKEM-976-AES', 'FrodoKEM-976-SHAKE', 'FrodoKEM-1344-AES', 'FrodoKEM-1344-SHAKE']


In [3]:
time_all = []

In [4]:
kemalgs = ["Kyber512", "BIKE-L1"]

for kemalg in kemalgs:
    
    time_enc = []
    time_dec = []

    reps = 100

    with oqs.KeyEncapsulation(kemalg) as client:
        #print("\nKey encapsulation details:")
        #print(client.details)
        
        client_public_key = client.generate_keypair()

        with oqs.KeyEncapsulation(kemalg) as server:

            for i in range(0,reps,1):
                tic = time.perf_counter()
                ciphertext, shared_secret_server = server.encap_secret(client_public_key)
                toc = time.perf_counter()
                time_enc.append(toc - tic)
        
            time_enc = mean(time_enc)
        
            for i in range(0,reps,1):
                tic = time.perf_counter()
                shared_secret_client = client.decap_secret(ciphertext)
                toc = time.perf_counter()
                time_dec.append(toc - tic)
            
            time_dec = mean(time_dec)
    
    time_all.append([kemalg, time_enc, time_dec])
    print(f"alg = {kemalg}, sig = {time_enc}, ver = {time_dec}")

alg = Kyber512, sig = 0.0003749827701540198, ver = 0.0005363655499604647
alg = BIKE-L1, sig = 0.0010247205500490963, ver = 0.01513207271000283


In [5]:
kemalgs = ["RSA-PKCS"]

for kemalg in kemalgs:
    
    time_enc = []
    time_dec = []

    reps = 100
    
    recipient_key = RSA.generate(1024)
    session_key = get_random_bytes(16)
    
    cipher_rsa = PKCS1_OAEP.new(recipient_key)
                
    for i in range(0,reps,1):
        tic = time.perf_counter()
        enc_session_key = cipher_rsa.encrypt(session_key)
        toc = time.perf_counter()
        time_enc.append(toc - tic)
        
    time_enc = mean(time_enc)
        
    for i in range(0,reps,1):
        tic = time.perf_counter()
        dec_session_key = cipher_rsa.decrypt(enc_session_key)
        toc = time.perf_counter()
        time_dec.append(toc - tic)
            
    time_dec = mean(time_dec)
        
    time_all.append([kemalg, time_enc, time_dec])
    print(f"alg = {kemalg}, sig = {time_enc}, ver = {time_dec}")

alg = RSA-PKCS, sig = 0.0021085112502987613, ver = 0.0065564135799650106


In [6]:
print(tabulate(time_all, headers = ["Algs", "Encaps", "Decaps"]))

Algs           Encaps       Decaps
--------  -----------  -----------
Kyber512  0.000374983  0.000536366
BIKE-L1   0.00102472   0.0151321
RSA-PKCS  0.00210851   0.00655641
